In [48]:
import torch
import torch.nn as nn

torch.manual_seed(777)
idx2char = ['h', 'e', 'l', 'o', 'w', 'r', 'd']
x_data = [0, 1, 2, 2, 3, 4, 5, 2, 3, 6]   # helloworld
one_hot_lookup = [[1, 0, 0, 0, 0, 0, 0],  # 0
                  [0, 1, 0, 0, 0, 0, 0],  # 1
                  [0, 0, 1, 0, 0, 0, 0],  # 2
                  [0, 0, 0, 1, 0, 0, 0],  # 3
                  [0, 0, 0, 0, 1, 0, 0],  # 4
                  [0, 0, 0, 0, 0, 1, 0],  # 5
                  [0, 0, 0, 0, 0, 0, 1]]  # 6
y_data = [1, 0, 2, 2, 3, 4, 5, 2, 3, 6]    # helloworld
x_one_hot = [one_hot_lookup[x] for x in x_data]
inputs = torch.Tensor(x_one_hot).unsqueeze(0)
labels = torch.LongTensor(y_data).view(-1)

input_size = 7
hidden_size = 7
num_layers = 1
batch_size = 1
sequence_length = len(x_data)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(num_layers, x.size(0), hidden_size)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        out = self.fc(out)

        return out.view(-1, input_size)

# Instantiate the model
model = Model()

# Set loss and optimizer function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
for epoch in range(1000):
    optimizer.zero_grad()
    loss = 0

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs,torch.LongTensor(labels))
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Print progress
    if (epoch+1) % 10 == 0:
        _, idx = outputs.max(1)
        idx = idx.squeeze().tolist()
        print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1, 1000, loss.item()))
        print("Predicted string: ", end='')
        print("".join([idx2char[idx[i]] for i in range(sequence_length)]))
        print("="*50)

print("Learning finished!")

Epoch [10/1000], Loss: 1.7172
Predicted string: hholowoloh
Epoch [20/1000], Loss: 1.3477
Predicted string: ohllowrloh
Epoch [30/1000], Loss: 1.0123
Predicted string: ohllowrlow
Epoch [40/1000], Loss: 0.7485
Predicted string: ehllowrlod
Epoch [50/1000], Loss: 0.5287
Predicted string: ehllowrlod
Epoch [60/1000], Loss: 0.3446
Predicted string: ehllowrlod
Epoch [70/1000], Loss: 0.2137
Predicted string: ehllowrlod
Epoch [80/1000], Loss: 0.1368
Predicted string: ehllowrlod
Epoch [90/1000], Loss: 0.0951
Predicted string: ehllowrlod
Epoch [100/1000], Loss: 0.0713
Predicted string: ehllowrlod
Epoch [110/1000], Loss: 0.0565
Predicted string: ehllowrlod
Epoch [120/1000], Loss: 0.0466
Predicted string: ehllowrlod
Epoch [130/1000], Loss: 0.0395
Predicted string: ehllowrlod
Epoch [140/1000], Loss: 0.0341
Predicted string: ehllowrlod
Epoch [150/1000], Loss: 0.0300
Predicted string: ehllowrlod
Epoch [160/1000], Loss: 0.0267
Predicted string: ehllowrlod
Epoch [170/1000], Loss: 0.0240
Predicted string: 

In [57]:
import torch

input_data = "hellwoorld"
label_data = "helloworld"
char_set = list(set(input_data))
char_dic = {w: i for i, w in enumerate(char_set)}
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
learning_rate = 0.1
input_dim = len(input_data)
x_data = [char_dic[c] for c in input_data]
y_data = [char_dic[c] for c in label_data]
x_one_hot = torch.nn.functional.one_hot(torch.tensor(x_data), num_classes=num_classes).float()
y_tensor = torch.tensor(y_data).long()

class RNNModel(torch.nn.Module):
    def __init__(self, hidden_size, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.rnn = torch.nn.RNN(input_size=data_dim, hidden_size=hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out

model = RNNModel(hidden_size=hidden_size, num_classes=num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in range(30):
    optimizer.zero_grad()
    output = model(x_one_hot.unsqueeze(0))
    loss = criterion(output.view(-1, num_classes), y_tensor.view(-1))
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    pred = output.argmax(dim=2)
    correct = pred.squeeze() == y_tensor
    accuracy = correct.float().mean()

    pred_str = ''.join([char_set[c] for c in pred.squeeze().tolist()])
    print(f"Epoch [{i+1}/30], Loss: {loss.item():.4f}, Accuracy: {accuracy.item():.2f}, Prediction: {pred_str}")


Epoch [1/30], Loss: 2.0149, Accuracy: 0.20, Prediction: heheheeeee
Epoch [2/30], Loss: 1.6611, Accuracy: 0.30, Prediction: llllllllll
Epoch [3/30], Loss: 1.5260, Accuracy: 0.30, Prediction: llllllllll
Epoch [4/30], Loss: 1.3058, Accuracy: 0.60, Prediction: llllololld
Epoch [5/30], Loss: 1.0690, Accuracy: 0.90, Prediction: holloworld
Epoch [6/30], Loss: 0.8090, Accuracy: 0.90, Prediction: holloworld
Epoch [7/30], Loss: 0.5591, Accuracy: 1.00, Prediction: helloworld
Epoch [8/30], Loss: 0.3639, Accuracy: 1.00, Prediction: helloworld
Epoch [9/30], Loss: 0.2336, Accuracy: 1.00, Prediction: helloworld
Epoch [10/30], Loss: 0.1489, Accuracy: 1.00, Prediction: helloworld
Epoch [11/30], Loss: 0.0938, Accuracy: 1.00, Prediction: helloworld
Epoch [12/30], Loss: 0.0590, Accuracy: 1.00, Prediction: helloworld
Epoch [13/30], Loss: 0.0380, Accuracy: 1.00, Prediction: helloworld
Epoch [14/30], Loss: 0.0257, Accuracy: 1.00, Prediction: helloworld
Epoch [15/30], Loss: 0.0183, Accuracy: 1.00, Prediction: 